# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [64]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-movie1-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Film Genre and directors explorative search") 

Consider the following exploratory information need:

> investigate the results concerning the different film genre over years and the directors for the cinema.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q25089`| Woody Allen       | node |
| `wd:Q3772`| Quentin Tarantino      | node |






Also consider

```
wd:Q25089 ?p ?obj .
```

is the BGP to retrieve all **properties of Woody Allen**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for films

2. Identify the BGP for director

3. Identify the BGP for film genre

4. Find how many western films are been released in each european country from 2010-01-01 to 2015-31-12 (the result set must be country IRI, label and #films).

5. Consider only films directed by Woody Allen or Quentin Tarantino. Then consider the date of publication of these films (take the first one if there are more than one date of publication). Return the year with more films release for the two director (the result set must be two elements and for each element there must be present the director IRI, the label of the IRI, the year and the number of films). 

6. Identify the BGP for Academy Awards 

7. Consider films directed by Woody Allen or Quentin Tarantino separately. Return the film which won more Academy Awards for the two directors (the result set must be two elements - one related to Allen and the other related to Tarantino -  and for each element there must be present the director IRI with its label, the film IRI with its label and the number of Academy awards won).

8. Consider the set of films directed by Allen or Tarantino. Return for each Academy Award the total number of nomination of these films (the result set must be a list of triples Academy Award IRI, label and #nomination).

In [65]:
## startup the evaluation
# setup the file and create the empty json
ipname = "m7.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 2_0


## Task 1

In [66]:
# properties incoming Allen
queryString = """
SELECT DISTINCT ?p ?name
WHERE { 
    ?obj ?p wd:Q25089.
    ?p sc:name ?name.
    FILTER(!isLiteral(?obj)).
} 
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('name', 'performer')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('name', 'father')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse')]
[('p', 'http://www.wikidata.org/prop/direct/P2650'), ('name', 'interested in')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('name', 'sibling')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child')]
[('p', 'http://www.

In [67]:
# use director to find the films
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    ?obj wdt:P57 wd:Q25089 .
    ?obj sc:name ?name.
} 
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q206124'), ('name', 'Midnight in Paris')]
[('obj', 'http://www.wikidata.org/entity/Q682262'), ('name', 'Alice')]
[('obj', 'http://www.wikidata.org/entity/Q14511869'), ('name', 'Magic in the Moonlight')]
[('obj', 'http://www.wikidata.org/entity/Q806092'), ('name', 'Bananas')]
[('obj', 'http://www.wikidata.org/entity/Q1004531'), ('name', 'Bullets Over Broadway')]
[('obj', 'http://www.wikidata.org/entity/Q1321004'), ('name', 'You Will Meet a Tall Dark Stranger')]
[('obj', 'http://www.wikidata.org/entity/Q971865'), ('name', "What's Up, Tiger Lily?")]
[('obj', 'http://www.wikidata.org/entity/Q729026'), ('name', 'Broadway Danny Rose')]
[('obj', 'http://www.wikidata.org/entity/Q733677'), ('name', 'Match Point')]
[('obj', 'http://www.wikidata.org/entity/Q740143'), ('name', 'Husbands and Wives')]
10


In [68]:
# instance of to find film
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    ?obj wdt:P57 wd:Q25089 ;
        wdt:P31 ?inst.
    ?inst sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('inst', 'http://www.wikidata.org/entity/Q506240'), ('name', 'television film')]
[('inst', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
[('inst', 'http://www.wikidata.org/entity/Q24862'), ('name', 'short film')]
[('inst', 'http://www.wikidata.org/entity/Q47467768'), ('name', 'operatic production')]
5


In [69]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q11424"
og_name = "film"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_0
The path is /locale/data/jupyter/prando/esw/tracks/2022/ground_truths/gt_json/workflow2_0.json
JSON object updated


`wd:Q11424` -> film

## Task 2

In [70]:
# properties of films
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    ?film wdt:P31 wd:Q11424;
        ?prop ?obj.
    ?prop sc:name ?name.
}
ORDER BY ?name
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P7003'), ('name', 'ACMI web ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P3593'), ('name', 'AFI Catalog of Feature Films ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P7118'), ('name', 'AMPAS collections film ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P6151'), ('name', 'ANICA ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P9032'), ('name', 'AVN movie ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P7194'), ('name', 'AZLyrics.com artist ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P9283'), ('name', 'AaRC title ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P6145'), ('name', 'Academy Awards Database film ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P6150'), ('name', 'Academy Awards Database nominee ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P7777'), ('name', 'AdoroCinema film ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P5083'), ('name', 'Adult Film Database 

In [71]:
#use the property P86 to list some composers
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    ?film wdt:P31 wd:Q11424;
        wdt:P86 ?obj.
    ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11720172'), ('name', 'Janusz Grudziński')]
[('obj', 'http://www.wikidata.org/entity/Q11767762'), ('name', 'Marcin Pospieszalski')]
[('obj', 'http://www.wikidata.org/entity/Q11817456'), ('name', 'Piotr Figiel')]
[('obj', 'http://www.wikidata.org/entity/Q13036911'), ('name', 'Aleksandër Lalo')]
[('obj', 'http://www.wikidata.org/entity/Q3299298'), ('name', 'Avni Mula')]
[('obj', 'http://www.wikidata.org/entity/Q4759658'), ('name', 'Andrzej Markowski')]
[('obj', 'http://www.wikidata.org/entity/Q7196950'), ('name', 'Piotr Hertel')]
[('obj', 'http://www.wikidata.org/entity/Q9285011'), ('name', 'Hajg Zaharian')]
[('obj', 'http://www.wikidata.org/entity/Q9312585'), ('name', 'Roman Rewakowicz')]
[('obj', 'http://www.wikidata.org/entity/Q159250'), ('name', 'Daniel Landa')]
[('obj', 'http://www.wikidata.org/entity/Q10287874'), ('name', 'Gabriel Migliori')]
[('obj', 'http://www.wikidata.org/entity/Q1692106'), ('name', 'Johan Zachrisson')]
[('obj', 

In [72]:
# look at their occupation
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    ?film wdt:P31 wd:Q11424;
        wdt:P86 ?obj.
    ?obj wdt:P106 ?inst.
    ?inst sc:name ?name.
    FILTER(REGEX(?name,"ompo")).
}
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q1415090'), ('name', 'film score composer')]
[('inst', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('inst', 'http://www.wikidata.org/entity/Q21680731'), ('name', 'opera composer')]
[('inst', 'http://www.wikidata.org/entity/Q21680663'), ('name', 'classical composer')]
[('inst', 'http://www.wikidata.org/entity/Q21680699'), ('name', 'operetta composer')]
[('inst', 'http://www.wikidata.org/entity/Q24256060'), ('name', 'professor of music composition')]
[('inst', 'http://www.wikidata.org/entity/Q64356038'), ('name', 'musical theatre composer')]
[('inst', 'http://www.wikidata.org/entity/Q8008937'), ('name', 'Category:Composers')]
[('inst', 'http://www.wikidata.org/entity/Q63536580'), ('name', 'video game composer')]
[('inst', 'http://www.wikidata.org/entity/Q5156675'), ('name', 'Compo')]
10


In [73]:
# look at their occupation
queryString = """
SELECT ?inst ?name
WHERE { 
    BIND(wd:Q36834 AS ?inst).
    ?inst sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
1


In [74]:
### insert the result of TASK 2 in the file
og_uri = "http://www.wikidata.org/entity/Q36834"
og_name = "composer"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_0
The path is /locale/data/jupyter/prando/esw/tracks/2022/ground_truths/gt_json/workflow2_0.json
JSON object updated


Find `wd:Q36834` -> composer

## Task 3

In [75]:
# use the property genre (P136) found some queries above
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    ?film wdt:P31 wd:Q11424;
        wdt:P136 ?obj.
    ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1054574'), ('name', 'romance film')]
[('obj', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama')]
[('obj', 'http://www.wikidata.org/entity/Q157443'), ('name', 'comedy film')]
[('obj', 'http://www.wikidata.org/entity/Q188473'), ('name', 'action film')]
[('obj', 'http://www.wikidata.org/entity/Q200092'), ('name', 'horror film')]
[('obj', 'http://www.wikidata.org/entity/Q2297927'), ('name', 'spy film')]
[('obj', 'http://www.wikidata.org/entity/Q369747'), ('name', 'war film')]
[('obj', 'http://www.wikidata.org/entity/Q52162262'), ('name', 'film based on literature')]
[('obj', 'http://www.wikidata.org/entity/Q645928'), ('name', 'biographical film')]
[('obj', 'http://www.wikidata.org/entity/Q846544'), ('name', 'disaster film')]
[('obj', 'http://www.wikidata.org/entity/Q52207399'), ('name', 'film based on a novel')]
[('obj', 'http://www.wikidata.org/entity/Q17013749'), ('name', 'historical film')]
[('obj', 'http://www.wikidata.org/en

In [76]:
# use the property genre (P136) found some queries above
queryString = """
SELECT DISTINCT ?inst ?name
WHERE { 
    ?film wdt:P31 wd:Q11424;
        wdt:P136 ?obj.
    ?obj wdt:P31 ?inst.
    ?inst sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q108329096'), ('name', 'genre by art form')]
[('inst', 'http://www.wikidata.org/entity/Q5937792'), ('name', 'crime fiction')]
[('inst', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
[('inst', 'http://www.wikidata.org/entity/Q1194240'), ('name', 'science fiction genre')]
[('inst', 'http://www.wikidata.org/entity/Q223393'), ('name', 'literary genre')]
[('inst', 'http://www.wikidata.org/entity/Q1792379'), ('name', 'art genre')]
[('inst', 'http://www.wikidata.org/entity/Q20076756'), ('name', 'speculative fiction genre')]
[('inst', 'http://www.wikidata.org/entity/Q108465955'), ('name', 'fiction genre')]
[('inst', 'http://www.wikidata.org/entity/Q15961987'), ('name', 'television genre')]
[('inst', 'http://www.wikidata.org/entity/Q7777573'), ('name', 'theatrical genre')]
[('inst', 'http://www.wikidata.org/entity/Q5151404'), ('name', 'comedic genre')]
[('inst', 'http://www.wikidata.org/entity/Q20656220'), ('name', 'thriller

In [77]:
# look at their occupation
queryString = """
SELECT ?inst ?name
WHERE { 
    BIND(wd:Q201658 AS ?inst).
    ?inst sc:name ?name.
}
"""

print("Results")
x=run_query(queryString)

Results
[('inst', 'http://www.wikidata.org/entity/Q201658'), ('name', 'film genre')]
1


In [78]:
### insert the result of TASK 3 in the file
og_uri = "http://www.wikidata.org/entity/Q201658"
og_name = "film genre"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_0
The path is /locale/data/jupyter/prando/esw/tracks/2022/ground_truths/gt_json/workflow2_0.json
JSON object updated


Found `wd:Q201658` -> film genre

## Task 4

Need to find date of release and also the country of release

In [79]:
# find the date and the country
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    ?film wdt:P31 wd:Q11424;
        ?prop ?obj.
    ?prop sc:name ?name.
    FILTER(REGEX(?name,"date") || REGEX(?name,"ountr") ).
}
"""

print("Results")
x=run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P577'), ('name', 'publication date')]
[('prop', 'http://www.wikidata.org/prop/direct/P495'), ('name', 'country of origin')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P2754'), ('name', 'production date')]
[('prop', 'http://www.wikidata.org/prop/direct/P3893'), ('name', 'public domain date')]
[('prop', 'http://www.wikidata.org/prop/direct/P2913'), ('name', 'date depicted')]
[('prop', 'http://www.wikidata.org/prop/direct/P1191'), ('name', 'date of first performance')]
[('prop', 'http://www.wikidata.org/prop/direct/P1619'), ('name', 'date of official opening')]
[('prop', 'http://www.wikidata.org/prop/direct/P6949'), ('name', 'announcement date')]
[('prop', 'http://www.wikidata.org/prop/direct/P3999'), ('name', 'date of official closure')]
[('prop', 'http://www.wikidata.org/prop/direct/P569'), ('name', 'date of birth')]
[('prop', 'http://www.wikidata.org/pro

In [80]:
# use the property publication date and country of origin. But first need to find United States
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    ?film wdt:P31 wd:Q11424;
        wdt:P495 ?obj.
    ?obj sc:name ?name.
    FILTER(REGEX(?name,"[U,u]")).
}
ORDER BY DESC (?name)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q42537'), ('name', 'flag of the United States')]
[('obj', 'http://www.wikidata.org/entity/Q165230'), ('name', 'flag of the Soviet Union')]
[('obj', 'http://www.wikidata.org/entity/Q380675'), ('name', 'cinema of the United States')]
[('obj', 'http://www.wikidata.org/entity/Q36704'), ('name', 'Yugoslavia')]
[('obj', 'http://www.wikidata.org/entity/Q41304'), ('name', 'Weimar Republic')]
[('obj', 'http://www.wikidata.org/entity/Q717'), ('name', 'Venezuela')]
[('obj', 'http://www.wikidata.org/entity/Q265'), ('name', 'Uzbekistan')]
[('obj', 'http://www.wikidata.org/entity/Q77'), ('name', 'Uruguay')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('obj', 'http://www.wikidata.org/entity/Q174193'), ('name', 'United Kingdom of Great Britain and Ireland')]
[('obj', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom')]
[('obj', 'http://www.wikidata.org/entity/Q170468'), ('name', 'United Arab Repu

In [81]:
# use the property publication date and country of origin. USA (Q30)
queryString = """
SELECT DISTINCT ?genre ?name COUNT(DISTINCT ?film) AS ?counts
WHERE { 
    ?film wdt:P31 wd:Q11424;
        wdt:P495 wd:Q30;
        wdt:P136 ?genre;
        wdt:P577 ?date.
    ?genre wdt:P31 wd:Q201658.
    FILTER ( isLiteral(?date) ) .
    FILTER(?date>=xsd:date("2010-01-01") && ?date<xsd:date("2016-01-01")).
    ?genre sc:name ?name.
}
GROUP BY ?genre ?name
ORDER BY DESC (?counts)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q93204'), ('name', 'documentary film'), ('counts', '1648')]
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama'), ('counts', '1584')]
[('genre', 'http://www.wikidata.org/entity/Q157443'), ('name', 'comedy film'), ('counts', '687')]
[('genre', 'http://www.wikidata.org/entity/Q200092'), ('name', 'horror film'), ('counts', '663')]
[('genre', 'http://www.wikidata.org/entity/Q188473'), ('name', 'action film'), ('counts', '573')]
[('genre', 'http://www.wikidata.org/entity/Q2484376'), ('name', 'thriller film'), ('counts', '521')]
[('genre', 'http://www.wikidata.org/entity/Q471839'), ('name', 'science fiction film'), ('counts', '318')]
[('genre', 'http://www.wikidata.org/entity/Q20442589'), ('name', 'LGBT-related film'), ('counts', '311')]
[('genre', 'http://www.wikidata.org/entity/Q859369'), ('name', 'comedy-drama'), ('counts', '290')]
[('genre', 'http://www.wikidata.org/entity/Q959790'), ('name', 'crime film'), ('counts', 

In [82]:
# use the property publication date and country of origin. USA (Q30)
queryString = """
SELECT DISTINCT ?genre ?name COUNT(DISTINCT ?film) AS ?counts
WHERE { 
    ?film wdt:P31 wd:Q11424;
        wdt:P495 wd:Q30;
        wdt:P136 ?genre;
        wdt:P577 ?date.
    ?genre wdt:P31 wd:Q201658.
    FILTER ( isLiteral(?date) ) .
    FILTER(?date>=xsd:date("1960-01-01") && ?date<xsd:date("1965-01-01")).
    ?genre sc:name ?name.
}
GROUP BY ?genre ?name
ORDER BY DESC (?counts)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('genre', 'http://www.wikidata.org/entity/Q130232'), ('name', 'drama'), ('counts', '339')]
[('genre', 'http://www.wikidata.org/entity/Q157443'), ('name', 'comedy film'), ('counts', '124')]
[('genre', 'http://www.wikidata.org/entity/Q200092'), ('name', 'horror film'), ('counts', '81')]
[('genre', 'http://www.wikidata.org/entity/Q860626'), ('name', 'romantic comedy'), ('counts', '81')]
[('genre', 'http://www.wikidata.org/entity/Q172980'), ('name', 'Western film'), ('counts', '77')]
[('genre', 'http://www.wikidata.org/entity/Q369747'), ('name', 'war film'), ('counts', '71')]
[('genre', 'http://www.wikidata.org/entity/Q842256'), ('name', 'musical film'), ('counts', '66')]
[('genre', 'http://www.wikidata.org/entity/Q471839'), ('name', 'science fiction film'), ('counts', '58')]
[('genre', 'http://www.wikidata.org/entity/Q959790'), ('name', 'crime film'), ('counts', '52')]
[('genre', 'http://www.wikidata.org/entity/Q1054574'), ('name', 'romance film'), ('counts', '50')]
10


In [83]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'),
#('obj', 'http://www.wikidata.org/entity/P495'), ('objname', 'country of origin')]
#[('class', 'http://www.wikidata.org/entity/Q11424'), ('classname', 'film')]
#[('result', 'http://www.wikidata.org/entity/Q201658'), ('resultname', 'film genre')]
#wdt:P31	instance of	predicate
#finding all film genres that contain western
#[('genre', 'http://www.wikidata.org/entity/Q172980'), ('genrename', 'Western film')]
#('obj', 'http://www.wikidata.org/entity/P577'), ('objname', 'publication date')]
#[('p', 'http://www.wikidata.org/prop/direct/P30'),
#('pname', 'continent'), ('n', 'http://www.wikidata.org/entity/Q5107'), ('name', 'continent')]
#[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentname', 'Europe')]

#AADING THE COUNTER  

queryString = """
SELECT DISTINCT   ?country  ?countryname (count (?film) AS ?numberOfFilms)
WHERE {
   # bind something
   ?film wdt:P136 wd:Q172980;
       wdt:P495 ?country;
       wdt:P577 ?date.
    ?country wdt:P30 wd:Q46. 
    
   # get the label
  
    ?country  sc:name ?countryname.

   
    FILTER (YEAR(?date)>2009 && YEAR(?date)<2016)
} 
GROUP BY ?country  ?countryname
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q34'), ('countryname', 'Sweden'), ('numberOfFilms', '2')]
[('country', 'http://www.wikidata.org/entity/Q221'), ('countryname', 'North Macedonia'), ('numberOfFilms', '1')]
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryname', 'Spain'), ('numberOfFilms', '1')]
[('country', 'http://www.wikidata.org/entity/Q43'), ('countryname', 'Turkey'), ('numberOfFilms', '1')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('countryname', 'Belgium'), ('numberOfFilms', '2')]
[('country', 'http://www.wikidata.org/entity/Q40'), ('countryname', 'Austria'), ('numberOfFilms', '3')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryname', 'United Kingdom'), ('numberOfFilms', '4')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryname', 'Russia'), ('numberOfFilms', '1')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryname', 'Denmark'), ('numberOfFilms', '2')]
[('country', 'http://www.wikidata.org/entit

In [84]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_0
The path is /locale/data/jupyter/prando/esw/tracks/2022/ground_truths/gt_json/workflow2_0.json
JSON object updated


## Task 5

In [85]:
# Finding films directed by Woody Allen and their details to have a clue.
# Sorting the result is just for make it easier to review.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT DISTINCT ?film ?pdate
WHERE {
   # bind something
  ?f wdt:P31 wd:Q11424 ;
     wdt:P57 wd:Q25089 ;
     wdt:P577 ?pdate .
   
   
   # get the label
   ?f sc:name ?film.

}
ORDER BY (?film)  (?pdate) 
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', "A Midsummer Night's Sex Comedy"), ('pdate', '1982-01-01T00:00:00Z')]
[('film', "A Midsummer Night's Sex Comedy"), ('pdate', '1982-10-15T00:00:00Z')]
[('film', 'A Rainy Day in New York'), ('pdate', '2019-07-26T00:00:00Z')]
[('film', 'A Rainy Day in New York'), ('pdate', '2019-09-18T00:00:00Z')]
[('film', 'A Rainy Day in New York'), ('pdate', '2019-12-05T00:00:00Z')]
[('film', 'A Rainy Day in New York'), ('pdate', '2019-12-12T00:00:00Z')]
[('film', 'Alice'), ('pdate', '1990-01-01T00:00:00Z')]
[('film', 'Alice'), ('pdate', '1991-02-28T00:00:00Z')]
[('film', 'Annie Hall'), ('pdate', '1977-04-20T00:00:00Z')]
[('film', 'Another Woman'), ('pdate', '1988-01-01T00:00:00Z')]
[('film', 'Another Woman'), ('pdate', '1989-03-16T00:00:00Z')]
[('film', 'Anything Else'), ('pdate', '2003-01-01T00:00:00Z')]
[('film', 'Bananas'), ('pdate', '1971-01-01T00:00:00Z')]
[('film', 'Bananas'), ('pdate', '1974-08-30T00:00:00Z')]
[('film', 'Blue Jasmine'), ('pdate', '2013-07-26T00:00:00Z')]
[('fi

In [86]:
# Lots of films have more than one publication date.
# Selecting first publication date for each film.

# Sorting the result is just for make it easier to review.

# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director  (Query 7)
# wdt:P577 => publication date

queryString = """
SELECT DISTINCT ?film (MIN(YEAR(?pdate)) as ?pYear)
WHERE {
   # bind something
  ?f wdt:P31 wd:Q11424 ;
     wdt:P57 wd:Q25089 ;
     wdt:P577 ?pdate .
   
   # get the label
   ?f sc:name ?film.

}
ORDER BY ASC(?pYear) (?film)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('film', "What's Up, Tiger Lily?"), ('pYear', '1966')]
[('film', 'Take the Money and Run'), ('pYear', '1969')]
[('film', 'Bananas'), ('pYear', '1971')]
[('film', 'Everything You Always Wanted to Know About Sex*'), ('pYear', '1972')]
[('film', 'Sleeper'), ('pYear', '1973')]
[('film', 'Love and Death'), ('pYear', '1975')]
[('film', 'Annie Hall'), ('pYear', '1977')]
[('film', 'Interiors'), ('pYear', '1978')]
[('film', 'Manhattan'), ('pYear', '1979')]
[('film', 'Stardust Memories'), ('pYear', '1980')]
[('film', "A Midsummer Night's Sex Comedy"), ('pYear', '1982')]
[('film', 'Zelig'), ('pYear', '1983')]
[('film', 'Broadway Danny Rose'), ('pYear', '1984')]
[('film', 'The Purple Rose of Cairo'), ('pYear', '1985')]
[('film', 'Hannah and Her Sisters'), ('pYear', '1986')]
[('film', 'Radio Days'), ('pYear', '1987')]
[('film', 'September'), ('pYear', '1987')]
[('film', 'Another Woman'), ('pYear', '1988')]
[('film', 'Crimes and Misdemeanors'), ('pYear', '1989')]
[('film', 'New York Stories

In [87]:
queryString = """

SELECT * WHERE
{ 
{
   select ?director ?directorname ?year ?number 
{
            VALUES ?director {wd:Q3772 } .
            ?director sc:name ?directorname.
    {
    select (count(?name) as?number ) ?year 
    {
    
    {
        SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
        WHERE {
            
           ?film wdt:P57 wd:Q3772;
            wdt:P577 ?date .
             ?film sc:name ?name  
            }group by  ?name  
            order by ?year
            limit 100
            
        } 
}group by ?year 
}
}order by desc(?number)
limit 1 
}
UNION
{
    select ?director ?directorname ?year ?number 
{
            VALUES ?director {wd:Q25089 } .
            ?director sc:name ?directorname.
    {
    select (count(?name) as?number ) ?year 
    {
    
    {
        SELECT DISTINCT (min(YEAR(?date) ) as ?year) ?name 
        WHERE {
            
           ?film wdt:P57 wd:Q25089;
            wdt:P577 ?date .
             ?film sc:name ?name  
            }group by  ?name  
            order by ?year
            limit 100
            
        } 
}group by ?year 
}
}order by desc(?number)
limit 1 
        
}
}group by ?year 
limit 2


"""
print("Results")
x=run_query(queryString)


Results
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('year', '2005'), ('number', '4')]
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('year', '1971'), ('number', '2')]
2


In [88]:
## more literals associated to the same element
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    year = i[2][1]
    films = i[3][1]
    for val in [year,films]:
        obj = {}
        obj["refers_to"] = f_uri
        obj["refers_to_name"] = f_name
        obj["check"] = "value"
        obj["value"]= val
        objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,"value", objs, elements_per_tuple = 2)

The index of this workflow is: 2_0
The path is /locale/data/jupyter/prando/esw/tracks/2022/ground_truths/gt_json/workflow2_0.json
JSON object updated


## Task 6

In [89]:
# write your final query

# Query 23


# wd:Q19020 => Academy Awards  (Query 2)

queryString = """
SELECT DISTINCT (wd:Q19020 as ?IRI) ?name
WHERE {
   # bind something
   ?s wdt:P31 wd:Q19020 .
   
   # get the label
   wd:Q19020 sc:name ?name.
}

LIMIT 1
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q19020'), ('name', 'Academy Awards')]
1


In [90]:
### insert the result of TASK 6 in the file
og_uri = "http://www.wikidata.org/entity/Q19020"
og_name = "Academy Awards"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 2_0
The path is /locale/data/jupyter/prando/esw/tracks/2022/ground_truths/gt_json/workflow2_0.json
JSON object updated


## Task 7

In [91]:
# write your queries

# Query 24

# Ckecking all films from both directors that recieve an Academy Awards.

# Sorting the result is just for make it easier to review.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """
SELECT DISTINCT ?Director ?fName ?aName
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   
   ?f wdt:P57 ?IRI ;
      wdt:P166 ?a .
      
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?f  sc:name ?fName.
   ?a sc:name ?aName.
   ?IRI sc:name ?Director.
}
ORDER BY ?Director ?fName ?aName
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Supporting Actor')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Writing, Original Screenplay')]
[('Director', 'Quentin Tarantino'), ('fName', 'Inglourious Basterds'), ('aName', 'Academy Award for Best Supporting Actor')]
[('Director', 'Quentin Tarantino'), ('fName', 'Once Upon a Time in Hollywood'), ('aName', 'Academy Award for Best Production Design')]
[('Director', 'Quentin Tarantino'), ('fName', 'Once Upon a Time in Hollywood'), ('aName', 'Academy Award for Best Supporting Actor')]
[('Director', 'Quentin Tarantino'), ('fName', 'Pulp Fiction'), ('aName', 'Academy Award for Best Writing, Original Screenplay')]
[('Director', 'Quentin Tarantino'), ('fName', 'The Hateful Eight'), ('aName', 'Academy Award for Best Original Score')]
[('Director', 'Woody Allen'), ('fName', 'Annie Hall'), ('aName', 'Academy Award for Best Actres

In [92]:


# Counting awards of each film, grouping based on director and film

# Sorting the result is just for make it easier to review.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """
SELECT DISTINCT ?Director (COUNT(DISTINCT(?aName)) as ?awards)
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   
   ?f wdt:P57 ?IRI ;
      wdt:P166 ?a .
      
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?f  sc:name ?fName.
   ?a sc:name ?aName.
   ?IRI sc:name ?Director.
}
GROUP BY ?Director ?fName
ORDER BY ?Director DESC(?awards)
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Director', 'Quentin Tarantino'), ('awards', '2')]
[('Director', 'Quentin Tarantino'), ('awards', '1')]
[('Director', 'Woody Allen'), ('awards', '4')]
[('Director', 'Woody Allen'), ('awards', '3')]
[('Director', 'Woody Allen'), ('awards', '1')]
5


In [93]:

# Maximun awards of each director

# Sorting the result is just for make it easier to review.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """
SELECT DISTINCT ?Director (MAX(?awards) as ?maxAwards) WHERE
{
   SELECT DISTINCT ?Director (COUNT(DISTINCT(?aName)) as ?awards) WHERE
   {
      # bind something
      VALUES ?IRI {wd:Q25089 wd:Q3772}
      
      ?f wdt:P57 ?IRI ;
         wdt:P166 ?a .
         
      ?a wdt:P31 wd:Q19020 .
      
      # get the label
      ?f  sc:name ?fName.
      ?a sc:name ?aName.
      ?IRI sc:name ?Director.
   }
   GROUP BY ?Director ?fName
}
GROUP BY ?Director

LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('Director', 'Quentin Tarantino'), ('maxAwards', '2')]
[('Director', 'Woody Allen'), ('maxAwards', '4')]
2


In [94]:

# Counting awards of each film, grouping based on director and film

# Sorting the result is just for make it easier to review.

# wd:Q206124 => 'Midnight in Paris'  (Query 2)
# wdt:P166 => award received (Query 7)
# wd:Q19020 => Academy Awards  (Query 2)
# wd:Q25089 => Woody Allen
# wd:Q3772 => Quentin Tarantino
# wdt:P57 => director 

queryString = """


SELECT DISTINCT ?IRI ?Director ?filmIRI ?film ?awards WHERE
{

############################################################################################################
# get the number of awards per film for each director
# Output are IRI, f (filmIRI) and awards
   {
   SELECT DISTINCT ?IRI ?f (COUNT(DISTINCT(?a)) as ?awards) WHERE
   {
      # bind something
      VALUES ?IRI {wd:Q25089 wd:Q3772}
      ?f wdt:P57 ?IRI ;
         wdt:P166 ?a .  
      ?a wdt:P31 wd:Q19020 .
   }
   GROUP BY ?IRI ?f
   }
############################################################################################################
# get the maximum of awards per film for each director
# Output are IRIm and maxAwards
   {
   SELECT DISTINCT ?IRIm (MAX(?awards) as ?maxAwards) WHERE
   {
      SELECT DISTINCT ?IRIm (COUNT(DISTINCT(?a)) as ?awards) WHERE
      {
         # bind something
         VALUES ?IRIm {wd:Q25089 wd:Q3772}
         ?f wdt:P57 ?IRIm ;
            wdt:P166 ?a .
         ?a wdt:P31 wd:Q19020 .
      }
      GROUP BY ?IRIm ?f
   }
   GROUP BY ?IRIm
   }
############################################################################################################

   FILTER(?IRI = ?IRIm && ?awards = ?maxAwards)

   # get the label
   ?f  sc:name ?film.
   ?IRI sc:name ?Director.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('IRI', 'http://www.wikidata.org/entity/Q3772'), ('Director', 'Quentin Tarantino'), ('film', 'Once Upon a Time in Hollywood'), ('awards', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q3772'), ('Director', 'Quentin Tarantino'), ('film', 'Django Unchained'), ('awards', '2')]
[('IRI', 'http://www.wikidata.org/entity/Q25089'), ('Director', 'Woody Allen'), ('film', 'Annie Hall'), ('awards', '4')]
3


In [95]:
#[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pname', 'properties for this type'), 
#('c', 'http://www.wikidata.org/entity/P57'), ('cname', 'director')]
#wd:Q25089	Woody Allen	node
#wd:Q3772	Quentin Tarantino	node
#[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')] 
#[('var', 'http://www.wikidata.org/entity/Q19020'), ('varname', 'Academy Awards')]

#FIRST APPROACH TO FIND THE ANSWER using union instead of value
queryString = """
SELECT * WHERE
{ 
{
   SELECT   ?director ?directorname ?film ?filmname (count(?awardname) as ?number ) 
    WHERE {
        VALUES ?director {  wd:Q25089 }
       ?film wdt:P57 ?director;
            wdt:P166 ?awards.
        ?awards wdt:P31 wd:Q19020 .
   
        ?film sc:name ?filmname  .
        ?awards sc:name ?awardname.
         ?director sc:name  ?directorname
     
        } group by  ?director ?directorname ?film ?filmname 
        order by desc(?number) 
        LIMIT 1
}
UNION
{
    SELECT ?director ?directorname ?film ?filmname (count(?awardname) as ?number ) 
    WHERE {
     VALUES ?director { wd:Q3772  }
       
       ?film wdt:P57 ?director ;
            wdt:P166 ?awards.
        ?awards wdt:P31 wd:Q19020 .
   
        ?film sc:name ?filmname  .
        ?awards sc:name ?awardname.
         ?director sc:name  ?directorname
     
        } group by  ?director ?directorname ?film ?filmname 
        order by desc(?number) 
        LIMIT 1
        
}
}
"""
print("Results")
x=run_query(queryString)



Results
[('director', 'http://www.wikidata.org/entity/Q25089'), ('directorname', 'Woody Allen'), ('film', 'http://www.wikidata.org/entity/Q233464'), ('filmname', 'Annie Hall'), ('number', '4')]
[('director', 'http://www.wikidata.org/entity/Q3772'), ('directorname', 'Quentin Tarantino'), ('film', 'http://www.wikidata.org/entity/Q571032'), ('filmname', 'Django Unchained'), ('number', '2')]
2


In [96]:
## more elements mixed between URI and literals
objs=[]
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    dir_uri = i[2][1]
    dir_name = i[3][1]
    year = i[4][1]
    # add the year element
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"] = year
    objs.append(obj)
    # add the director element
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"] = dir_uri
    obj["name"] = dir_name
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_REFERRED ,["uri","name","value"], objs, elements_per_tuple = 2)

The index of this workflow is: 2_0
The path is /locale/data/jupyter/prando/esw/tracks/2022/ground_truths/gt_json/workflow2_0.json
JSON object updated


## Task 8

In [97]:

# Finding list of all films from directors nominated for any kind of Academy Awards to have a clue.
# Sorting the result is just for make it easier to review.

# wd:Q19020 => Academy Awards
# wdt:P1411 => nominated for  (Query 7)

queryString = """
SELECT DISTINCT ?Director ?fName ?aName
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   ?f wdt:P57 ?IRI ;
      wdt:P1411 ?a .
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?f  sc:name ?fName.
   ?a sc:name ?aName.
   ?IRI sc:name ?Director.
}
ORDER BY ?Director ?fName ?aName
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Cinematography')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Picture')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Sound Editing')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Supporting Actor')]
[('Director', 'Quentin Tarantino'), ('fName', 'Django Unchained'), ('aName', 'Academy Award for Best Writing, Original Screenplay')]
[('Director', 'Quentin Tarantino'), ('fName', 'Inglourious Basterds'), ('aName', 'Academy Award for Best Cinematography')]
[('Director', 'Quentin Tarantino'), ('fName', 'Inglourious Basterds'), ('aName', 'Academy Award for Best Director')]
[('Director', 'Quentin Tarantino'), ('fName', 'Inglourious Basterds'), ('aName', 'Academy Award for Best Film Editing')]
[('Director', 'Quentin Taran

In [98]:

# Counting no of nominated films for each Academic Awards.
# Sorting the result is just for make it easier to review.

# wd:Q19020 => Academy Awards
# wdt:P1411 => nominated for  (Query 7)

queryString = """
SELECT DISTINCT  ?aName (COUNT(?f) as ?noOfFilm)
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   ?f wdt:P57 ?IRI ;
      wdt:P1411 ?a .
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?a sc:name ?aName.
}
GROUP BY ?aName
ORDER BY ?aName
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('aName', 'Academy Award for Best Actor'), ('noOfFilm', '3')]
[('aName', 'Academy Award for Best Actress'), ('noOfFilm', '3')]
[('aName', 'Academy Award for Best Cinematography'), ('noOfFilm', '4')]
[('aName', 'Academy Award for Best Costume Design'), ('noOfFilm', '2')]
[('aName', 'Academy Award for Best Director'), ('noOfFilm', '9')]
[('aName', 'Academy Award for Best Film Editing'), ('noOfFilm', '3')]
[('aName', 'Academy Award for Best Original Score'), ('noOfFilm', '1')]
[('aName', 'Academy Award for Best Picture'), ('noOfFilm', '6')]
[('aName', 'Academy Award for Best Production Design'), ('noOfFilm', '5')]
[('aName', 'Academy Award for Best Sound'), ('noOfFilm', '1')]
[('aName', 'Academy Award for Best Sound Editing'), ('noOfFilm', '2')]
[('aName', 'Academy Award for Best Supporting Actor'), ('noOfFilm', '7')]
[('aName', 'Academy Award for Best Supporting Actress'), ('noOfFilm', '11')]
[('aName', 'Academy Award for Best Writing, Original Screenplay'), ('noOfFilm', '19')]


In [99]:
# Sorting the result is just for make it easier to review.

# wd:Q19020 => Academy Awards
# wdt:P1411 => nominated for  (Query 7)

queryString = """
SELECT DISTINCT (?a as ?awardIRI) ?award (COUNT(?f) as ?noOfNominates)
WHERE {
   # bind something
   VALUES ?IRI {wd:Q25089 wd:Q3772}
   ?f wdt:P57 ?IRI ;
      wdt:P1411 ?a .
   ?a wdt:P31 wd:Q19020 .
   
   # get the label
   ?a sc:name ?award.
}
GROUP BY ?a ?award
ORDER BY ?award
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('awardIRI', 'http://www.wikidata.org/entity/Q103916'), ('award', 'Academy Award for Best Actor'), ('noOfNominates', '3')]
[('awardIRI', 'http://www.wikidata.org/entity/Q103618'), ('award', 'Academy Award for Best Actress'), ('noOfNominates', '3')]
[('awardIRI', 'http://www.wikidata.org/entity/Q131520'), ('award', 'Academy Award for Best Cinematography'), ('noOfNominates', '4')]
[('awardIRI', 'http://www.wikidata.org/entity/Q277536'), ('award', 'Academy Award for Best Costume Design'), ('noOfNominates', '2')]
[('awardIRI', 'http://www.wikidata.org/entity/Q103360'), ('award', 'Academy Award for Best Director'), ('noOfNominates', '9')]
[('awardIRI', 'http://www.wikidata.org/entity/Q281939'), ('award', 'Academy Award for Best Film Editing'), ('noOfNominates', '3')]
[('awardIRI', 'http://www.wikidata.org/entity/Q488651'), ('award', 'Academy Award for Best Original Score'), ('noOfNominates', '1')]
[('awardIRI', 'http://www.wikidata.org/entity/Q102427'), ('award', 'Academy Award for

In [100]:
## single literal associated to an URI
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    val = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= val
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"8", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 2_0
The path is /locale/data/jupyter/prando/esw/tracks/2022/ground_truths/gt_json/workflow2_0.json
JSON object updated
